# Модули

## Какво е модул?

Всеки един Python файл (.py) на практика е модул. Освен това е възможно библиотека, написана на С, и вмъкната динамично също да бъде модул. Третия тип модули са вградените в езика такива.

В тази лекция се фокусираме върху първия тип и модули и как можем да ги създаваме, вмъкваме и боравим с тях.

## Как да създам модул?

Казахме, че всеки Python файл е валиден модул. 

Нека създадем един такъв с няколко дефиниции вътре (в папката с тази тетрадка вече би трябвало да се съдържа файл `hitchhikers.py`).

## `import`

Имената, функциите и класовете, които създадохме в този файл, не могат да бъдат достъпени директно от друг файл:

In [ ]:
compute()

Можем обаче да ги вмъкнем в друг файл (модул) чрез `import {името_на_модула}` (името на файла преди разширението `.py` се превръща в име на модула):

In [ ]:
import hitchhikers

hitchhikers.compute()

`import` освен, че интерпретира целия код на модула, добавя имената и дефинициите в един обект от тип модул, имащ името на модула. Затова и ги достъпваме чрез `името_на_модула.име_на_обекта`.

Какво се съдържа в един модул можем лесно да видим с `dir()`:

In [ ]:
dir(hitchhikers)

In [ ]:
hitchhikers.ANSWER

In [ ]:
computer = hitchhikers.TheGreatDeepThought()
computer.ask()

In [ ]:
hitchhikers.__name__

In [ ]:
dir()  # by default it shows the contents of the *current* module

In [ ]:
__name__

## Добре, обаче `import` къде точно търси?

1. Директорията, в която се намира Python скрипта, който се изпълнява (или текущата, ако интерпретаторът е пуснат интерактивно)
2. Директориите, които са описани в `PYTHONPATH` променливата на средата
3. Лист от директории, зададен по време на инсталацията на Python

Този списък от възможни директории може да се види със `sys.path`:

In [ ]:
import sys
sys.path

## Варианти на `import`

С `from {module} import {something}, {something_else}, ...` можем да импортираме само определени имена от модула, като те биват добавени към съдържанието на текущия (т.е. достъпваме ги без името на оригиналния модул и точка отпред):

In [ ]:
from hitchhikers import compute
compute()

С `from {module} import {something} as {alias}, {something_else} as {other_alias}, ...` можем да прекръстим импортираните имена:

In [ ]:
from hitchhikers import ANSWER, TheGreatDeepThought as Computer
comp = Computer()
comp.ask() == ANSWER

In [ ]:
"ANSWER" in dir()

Ако искаме абсолютно всички имена на вмъкнем и ползваме в текущия модул по този начин (без тези, започващи с подчертавка `_`), можем да използваме астериск `*`:

In [ ]:
# изпълни тази клетка ако си изпълнил горните, за да се зачистят import-ите
del hitchhikers, ANSWER, TheGreatDeepThought, compute, Computer

In [ ]:
from hitchhikers import *

compute() == ANSWER

Лимитация на астерикс синтаксиса е, че не може да използва в блок (може само на най-външното ниво на модула):

In [ ]:
del compute, ANSWER, TheGreatDeepThought

In [ ]:
def obtain_answer():
    from hitchhikers import *
    return compute()

Както казахме, по подразбиране from {module} import * вмъква абсолютно всички имена от `module`, които не започват с подчертавка. Имаме всъщност контрол над това, кое може да се вмъкне чрез астерикс, като дефинираме `__all__` във въпросния модул. Стойността му е лист от всички имена, които ще бъдат вмъкнати от `*`.

*Пример*:

След добавяне на
```python
__all__ = ['compute', 'TheGreatDeepThought']
```
в `hitchhikers.py`, следният код, изпълнен в `script.py` (в същата директория) ще хвърли `NameError`:
```python
from hitchhikers import *
print(hitchhikers.ANSWER)  # 💥
```

## Пакети

Пакет e набор от модули. За Python всяка директория, в която има модули, се превръща в пакет (package).

*Note*: Във версии по-ранни от Python 3.3 трябва задължително в директорията да има файл с име `__init__.py`.

В директорията на тетрадката би трябвало да има папка `game`, съдържаща няколко файла и папки:

In [ ]:
!ls -l game

В горния пример `game` е пакет, съдържащ модулите `engine`, `level` и `player`. Освен тях, той съдържа и подпакетът `players`.

Ако искаме да вмъкнем някой модул от пакета, можем да го направим чрез името на пакета (или всички пакети по веригата, разделени с точка), последвано от точка и името на въпросния модул:

In [ ]:
import game.level

game.level.EASY

In [ ]:
import game.players.ai

game.players.ai.AI(10)

Вече въведените по-горе синтактични варианти на `import` също важат:

In [ ]:
from game.players.ai import AI
from game.level import EASY as easy, MEDIUM as medium, HARD as hard
from game.engine import *

Освен това, можем и да вмъкнем модули чрез `from {package} import {module} [as {alias}], ...`:

In [ ]:
from game import level, engine

print(level.EASY)
print(engine.GameState)

## `__init__.py`

На теория можем и да импортнем само пакета. По подразбиране това няма да добави нови модули и имена:

In [ ]:
del game.level, game.player, game.engine  # зачисти тетрадката от предните импорти

In [ ]:
import game

game.level  # 💥

Ако искаме да добавим и модули от пакета при импортирането му, можем да ги импортнем в `__init__.py`, намиращ се в директорията на пакета.

Т.е. ако в `game/__init__.py` имаме:
```python
import game.engine, game.level, game.player
```

то можем да импортнем пакета `game` и да използваме всички модули от него:
```python
# в друг файл, извън пакета `game`:
import game
print(game.level.EASY)  # no error
```

В `__init__.py` можем да напишем какъвто искаме инициализационен код, глобален за всички модули в пакета. Съдържанието на скрипта се изпълнява веднага при импортиране на пакета.

Както при модулите, така и тука можем да дефинираме поведението на `from {package} import *` чрез `__all__`. По подразбиране, както видяхме за `import {package}`, това е празен списък, т.е. нищо няма да се вмъкне (за разлика от поведението при модулите, когато се вмъква абсолютно всяко име от модула, което не започва с подчертавка).

Т.е. ако напишем в `game/__init__.py`:

```python
__all__ = ["engine", "level", "player"]
```

то ще можем:

```python
# в друг файл, извън пакета `game`:
from game import *
print(level.EASY)  # no error
```

## Релативни импорти

Дотук разгледахме примерни за **абсолютни** импорти, т.е. достъпът до даден модул от рамките на пакета или извън него става през пътя от пакета до модула, например `game.players.ai` достъпва модулът `ai` от пакета `players` в пакета `game`.


In [ ]:
import game.players.ai

Със значението на `.` и `..` от Unix файловата система, можем да използваме същите тези символи за **релативни** импорти в Python. Те се оценяват спрямо локацията на `import` statement-a.

Например, във файла `game/players/input_player.py` ни трябва `player` модула от пакета `game`. Можем да го направим по абсолютен и релативен начин:

```python
from game import player  # абсолютен импорт
```

```python
from .. import player  # релативен импорт
```

* `..` означава "пакетът, намиращ се над текущия".
* `..pkg` означва модулът/пакетът `pkg` от пакетът, намиращ се над текущия.

Например:
```python
from ..player import Player
```
Ще вмъкне името `Player` от модула `player` от пакета, намиращ се над текущия.

* `.` означава "текущия пакет".
* `.pkg` означава модулът/пакетът `pkg` от текущия пакет.

Релативните импорти имат недостатъка обаче, че зависят от местоположението на `import`-a. Освен това в скриптове (т.е. изпълним код, който не е вмъкнат чрез модул) имат различно поведение:

In [ ]:
from . import hitchhikers

## `if __name__ == "__main__"`

Както бяхме споменали, при импорт се изпълнява кода на съответния модул. Като пример за това можем да изведем философията на Python, намираща се във вградения модул `this`:

In [ ]:
import this

Текущо-изпълнимият файл/модул/скрипт за Python се казва `"__main__"`, т.е. неговия `__name__` е `"__main__"`:
```python

In [ ]:
__name__

Ако файлът не се изпълнява директно, а бъде импортнат от друг, то в неговия `__name__` ще е името на модула. Това означава, че можем да различим дали файлът се изпълнява директно или е импортнат. Полезно е в случаите, когато искаме да напишем примерно някакви тестове или демонстрации на модула, които да се изпълнят само ако го изпълним директно, и да не се изпълняват при всяко вмъкване. (разгледайте например `game/engine.py`)